# Домашнее задание 5. Линейные модели

In [1]:
import random as pr
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
import sklearn.metrics as sm
import sklearn.linear_model as lm
from scipy.spatial.distance import cityblock
from sklearn.ensemble import RandomForestClassifier
import math
import sys
import csv
import sklearn.naive_bayes as nb
from sklearn.linear_model import Ridge, RidgeClassifier

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = np.load("out_4_without_rts_and_replies.dat.npz")
#data = np.load("out_4_denis.dat.npz")
users = data["users"]
X_dataset = data["data"].reshape(1,)[0]

In [9]:
from sklearn import preprocessing
X_all = preprocessing.normalize(X_dataset.tocsc()[:, features_counts > 100].toarray(), axis=1)


In [5]:
TRAINING_SET_URL = "twitter_train.txt"
EXAMPLE_SET_URL = "twitter_example.txt"
df_users_train = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["uid", "cat"])
df_users_ex = pd.read_csv(EXAMPLE_SET_URL, sep=",", header=0, names=["uid", "cat"])
df_users_ex['cat'] = None
df_users = pd.concat([df_users_train, df_users_ex])
df_users = df_users.reset_index(drop=True)
df_users.head()

,uid,cat
0,40585382,1.0
1,30005269,0.0
2,3314758074,0.0
3,14311688,0.0
4,110457801,0.0


In [7]:
train_users = df_users_train['uid'].values
ix = np.in1d(users, train_users).reshape(users.shape)
X = X_all[np.where(ix)]
X.shape

(5000, 322133)

In [10]:
Y = df_users_train['cat'].values
print "Resulting training set: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y.shape[0])

Resulting training set: (5000x322133) feature matrix, 5000 target vector


In [8]:
def draw_log_hist(x):
    """Draw tokens histogram in log scales"""
    freq = np.array(((x!=0).sum(axis = 0)+1).tolist()[0])
    ran = xrange(1, x.shape[1]+1)
    #plt.figure(figsize=(5, 5))
    #plt.xscale('log')
    #plt.yscale('log')
    #plt.scatter(ran, sorted(freq, reverse=True))
    return freq

features_counts = draw_log_hist(X)

Проведем отбор признаков. В самом простом случае просто удаляем признаки, имеющие ненулевое значение у менее, чем 100 пользователей.

In [14]:
X1 = X.tocsc()[:, features_counts > 10].toarray()
X1.shape

(5000L, 20737L)

In [15]:
ind = np.array([1 if X1[i].max()==0.0 else 0 for i in xrange(X1.shape[0])])
a = X[ind==0]
print a.shape

(4917, 322133)


In [12]:
Y_wo_zeros = Y.copy()
Y_wo_zeros = Y[ind==0]
Y_wo_zeros.shape

(4917L,)

In [16]:
#from sklearn import preprocessing
#X1_norm_wo_zeros = preprocessing.normalize(a, axis=1)
#X1_norm = preprocessing.normalize(X1, axis=1)
X_wo_zeros = X1[ind==0]

In [18]:
X_wo_zeros.shape


(4917L, 20737L)

In [ ]:
X_all.shape

In [21]:
def fit_and_predict(model, x_train, x_test, y_train, y_test ):
    fit = model.fit(x_train, y_train)
    y_pred = fit.predict(x_test)
    clusters = []
    #for i in y_pred:
    #    if i>=0.5:
    #        clusters.append(1)
    #    else:
    #        clusters.append(0)
    return sm.roc_auc_score(y_test, y_pred)

In [19]:
x_train, x_test, y_train, y_test = cv.train_test_split(X_wo_zeros, Y_wo_zeros, test_size=0.3)
model = Ridge(alpha=50.0, fit_intercept=True, normalize=False, solver='sparse_cg')
fit = model.fit(x_train, y_train)
y_pred = fit.predict(x_test)
print sm.roc_auc_score(y_test, y_pred)

0.565519572117


In [ ]:
from sklearn.cross_validation import KFold
from sklearn.feature_selection import RFECV
rfecv = RFECV(estimator=model, step=50, cv=KFold(len(Y), 2), scoring='roc_auc')
rfecv.fit(X1_norm, Y)
print rfecv

In [ ]:
C = [0.005, 0.01, 0.015, 0.017, 0.0195, 0.02, 0.023, 0.025, 0.5, 0.7, 0.9, 1.0, 10.0, 50.0, 100.0, 1000.0, 10000.0, 50000.0, \
     100000.0, 500000.0, 750000.0, 1000000.0]
results = []
x_train, x_test, y_train, y_test = cv.train_test_split(X_wo_zeros, Y_wo_zeros, test_size=0.3)#X1_norm_wo_zeros
for i, c in enumerate(C):
    model = lm.LogisticRegression(tol=0.00001, C=c, penalty='l2', fit_intercept=True, solver='newton-cg')
    #model = Ridge(alpha=c, fit_intercept=True, normalize=False, solver='sag')
    acc = fit_and_predict(model,  x_train, x_test, y_train, y_test)
    #if acc>0.65 :
    print "C = %.3f; Acc = %.3f" % (c, acc)
    #results.append(acc)
#freqs[np.argmax(results)]+=1
#print "Maximum Acc = %.3f by C = %.3f" % (max(results), C[np.argmax(results)])
#print freqs

C = 0.005; Acc = 0.607


In [ ]:
tpr, fpr, roc_auc = classify(X1, Y, 0.3, 1.0)

print "Area under the ROC curve : %f" % roc_auc
"""
def plot_roc_curve(tpr, fpr, roc_auc): 
    """Plot ROC curve"""
    # Plot ROC curve
    plt.figure(figsize=(4,4))
    plt.fill_between(fpr, tpr, alpha=0.5)
    plt.plot([0, 1], [0, 1])
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    
plot_roc_curve(tpr, fpr, roc_auc)
"""

In [ ]:
ex_users = df_users_ex['uid'].values
ix_ex = np.in1d(users, ex_users).reshape(users.shape)
X_ex = X_all[np.where(ix_ex)]

In [ ]:
X_ex.shape

In [ ]:
X1_ex = X_ex.tocsc()[:, features_counts > 100].toarray()

In [ ]:
X1_ex.shape

In [ ]:
#X1_ex_norm = preprocessing.normalize(X1_ex, axis=1)
X1_ex_norm.shape

In [ ]:
#model = lm.LogisticRegression(tol=0.0001, C=0.9, penalty='l1', fit_intercept=True, solver='liblinear')
#lm.LogisticRegression(tol=0.0001, C=0.017, penalty='l1')
#model = Ridge(alpha=50.0, fit_intercept=True, normalize=False, solver='sparse_cg')
model = lm.LogisticRegression(tol=0.00001, C=0.5, penalty='l2', fit_intercept=True, solver='newton-cg')
fit = model.fit(X_wo_zeros, Y_wo_zeros)

In [ ]:
y_prob = fit.predict(X_ex)
print y_prob

In [ ]:
clusters = []
for i in y_prob:
    if i>=0.5:
        clusters.append(1)
    else:
        clusters.append(0)

In [ ]:
df_users_ex['cat'] = clusters
df_users_ex.head()

In [ ]:
df_users_ex.to_csv("respond.csv", sep=",", index=False, encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
#Попробовать добавить оценку эмоционального окраса твитов